# Credit mining

A Job can generate new credits with time, by bonding Keep3r Liquidity Pool tokens kLP to it. Liquidities will be handled by the Keep3r Liquidity Pools (`keeep3r.pool`).
Once kLPs are added to a job with addLiquidityToJob, the job starts immediately to mint new KP3R credits, that can be collectable only by the keepers, in reward for working the job. The credit minting system requires no further action from the job owner (`provider`).

### Setup
Run `00-setup.ipynb` and select its kernel

In [7]:
// add traces of interest
next(async() =>{
//     await snapshots.revert(setupSnap)
    await $.resetRecording()
    $.resetTraces()

    $.setPeriodTrace(432000)
    
    $.addViewTrace(keep3r.v2, 'jobLiquidityCredits', job.address)
    $.addViewTrace(keep3r.v2, 'totalJobCredits', job.address)
    $.addViewTrace(keep3r.v2, 'jobPeriodCredits', job.address) 
//     $.addViewTrace(keep3r.v1, 'bonds', keep3r.keeper.address) 
    $.addEventTrace(keep3r.v2.web3, 'LiquidityCreditsReward', '_rewardedAt')
    $.addEventTrace(keep3r.v2.web3, 'LiquidityAddition')
    $.addEventTrace(keep3r.v2.web3, 'KeeperWork')
})

tickQuote 0.11163838045095273


### Reward Periods

To handle KP3R credits minting and quoting, Keep3r introduces reward periods, in which KP3R quote remains stable for each pair, and gas-efficiently processed. These quotes are used within the protocol to mint credits and reward keepers.

The underlying KP3R of the liquidity provided, should generate the same amount of KP3R every inflationPeriod, thereby minting the proportional amount each rewardPeriod as KP3R credits for the job.

In [8]:
// liquidity added in 00-setup

next(async()=>{
    
    rewardPeriodTime = (await keep3r.v2.rewardPeriodTime()).toNumber()
    inflationPeriod = (await keep3r.v2.inflationPeriod()).toNumber()
    
    console.log('KP3R spent', bnToNumber(kp3rInitialBalance.sub(kp3rBalance)))
    console.log('kLP minted', bnToNumber(klpBalance))
    console.log('reward period',  rewardPeriodTime / (3600 * 24), 'days')
    console.log('inflation period', (await keep3r.v2.inflationPeriod()).toNumber() / (3600 * 24), 'days')
    console.log('KP3R minted by reward period', bnToNumber(await keep3r.v2.jobPeriodCredits(job.address)))
    console.log('KP3R minted by inflation period', bnToNumber((await keep3r.v2.jobPeriodCredits(job.address)).mul(inflationPeriod).div(rewardPeriodTime)))
})

KP3R spent 9.99
WETH spent 1.09
kLP minted 3.3
totalJobCredits 0
jobPeriodCredits 1.39
jobLiquidityCredits 0
KP3R spent 9.99
kLP minted 3.3
reward period 5 days
inflation period 34 days
KP3R minted by reward period 1.39
KP3R minted by inflation period 9.45


To determine the value of a certain liquidity, Keep3r uses a TWAP calculation to get the average quote of a pair in the last completed epoch. The same calculation is applied to quote rewards for keepers (that spend gas in ETH and receive KP3R rewards), using a predefined KP3R/WETH pool as an oracle.

- A job will mint the result of quoteLiquidity every rewardPeriodTime 
- Keep3r will store and use the average quote for the last epochfor each given liquidity
- Remaining credits will be updated to current quotes each time a `rewardPeriod` starts
- Credits older than previous `rewardPeriod` will no longer be updateable and will expire

In [9]:
// credit mining without working

next(async()=>{
    
    await $.sleepAndRecord(
        3 * rewardPeriodTime,
        $.time(4,'hours')
    )
    
    await $.draw()
})

In [10]:
// credit mining with twap change

next(async()=>{
    await $.resetRecording()
    
    await $.sleepAndExecute(
        3 * rewardPeriodTime,
        $.time(12,'hours'),
        async()=>{await common.makeASwap(provider, keep3r.v1.address, constants.WETH_ADDRESS, provider.address, 1000, toUnit(1000))},
        $.time(2,'days')
    )
    
    await $.draw()
})

In [10]:
// balanced work

next(async()=>{
    await $.sleep(rewardPeriodTime)
    await $.resetRecording()
        
    await $.sleepAndExecute(
        4 * rewardPeriodTime,
        $.time(4,'hours'),
        async()=>{await job.connect(keep3r.keeper).workHard(2)},
        $.time(36,'hours')
    )
    
    await $.draw()
})

In a normal case scenario, a job should be rewarded once every rewardPeriodStart and burn all remaining credits from last epoch. When jobs are running low on credits, they will be rewarded before the next rewardPeriod starts, and those credits will be minted with the previous epoch quote, so when next rewardPeriod starts, and liquidity quotes are updated, the job will have its remaining KP3R credits updated to the new quotes.

In [11]:
// intense work

next(async()=>{
    await $.sleep(rewardPeriodTime)
    await $.resetRecording()
    block = await $.block()
    await advanceTimeAndBlock(rewardPeriodTime - (block.timestamp % rewardPeriodTime))
    
    await $.sleepAndExecute(
        2 * rewardPeriodTime,
        $.time(4,'hours'),
        async()=>{await job.connect(keep3r.keeper).workHard(3)},
        $.time(12,'hours')
    )
 
    await $.draw()
})

On a deficitary job scenario, when the job is spending more KP3R in a period than it is rewarded, the job will be able to keep on paying keepers, but the minting period starts to shrink, having to mint more frequently, and rewarding extra credits for the keeper, every time he has to reward the job to get his payment.

Each time, the reward period will be shorter, as the job is not being rewarded the full amount for a period, but only the proportional relation of the time that passed since last reward, and the rewardPeriod.

Ultimately, when the job has not enough `totalJobCredits` to reward the keeper for working the job (and some extra to pay the keeper for rewarding the credits), the transaction will revert with `InsufficientFunds`.

In [19]:
// maximum credit spending

next(async()=>{
    await $.resetRecording()
    block = await $.block()
    await advanceTimeAndBlock(rewardPeriodTime - (block.timestamp % rewardPeriodTime))
    
    await $.sleepAndExecute(
        2 * Math.floor(1.9 * rewardPeriodTime),
        $.time(4,'hours'),
        async()=>{await job.connect(keep3r.keeper).workHard(25)},
        Math.floor(1.8 * rewardPeriodTime)
    )
 
    await $.sleepAndRecord(rewardPeriodTime, $.time(4,'hours'))
    
    await $.draw()
})

At particular times, a job can make a payment of up to 1.999 times its `jobPeriodCredits`, as long as all the credits minted have not yet expired.